BAIT 508 Group Project: Industry Analysis

PROJECT OVERVIEW: 
The goal of this project is to conduct an in-depth analysis of public US firms within selected industry sector(s) using various data analyses and natural language processing (NLP) techniques that we learned in BAIT 508. Each team will choose at least one industry sector to investigate and utilize multiple datasets to extract valuable industry insights from the data.

The project will utilize three datasets (located in the data folder):
•	public_firms.csv
•	major_groups.csv
•	2020_10K_item1_full.csv

In [ ]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
import seaborn as sns
import matplotlib.style as style
import string
from DocumentSimilarity import DocumentSimilarity
from collections import Counter
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [ ]:
filepath_data= r'C:\Users\sarda\Desktop\UBC\BAIT 508\Group_Project_BAIT508\data'
public_firms=pd.read_csv(r'C:\Users\sarda\Desktop\UBC\BAIT 508\Group_Project_BAIT508\data\public_firms.csv')
twenty20_10K_item1_full=pd.read_csv(r'C:\Users\sarda\Desktop\UBC\BAIT 508\Group_Project_BAIT508\data\2020_10K_item1_full.csv')
major_groups =pd.read_csv(r'C:\Users\sarda\Desktop\UBC\BAIT 508\Group_Project_BAIT508\data\major_groups.csv')

Part 1. Quantitative Analysis of the Industry Sector

A.	[Industry Sector Selection and Data Filtering; 20 points] 

1.	The file "data/major_groups.csv" contains a list of major industry sectors and their corresponding codes (column "major_group"). Your first task is to choose at least one industry sector that interests your group. It is okay if multiple groups choose the same industry sector, so you don’t need to coordinate with other groups.


In [ ]:
agri = ( '1','2','7','8','9') #Agriculture and Related Industries
wholesale = ( '50','51') #Wholesale
food =('54') #Food
busi_serv = ('73') #BusinessSerivices
wholesale_only = ('50') #Wholesale_only

2.	Next, filter the data in "data/public_firms.csv" to only include the firms belonging to the industry sector(s) you have selected. You can use the "major_group" value, which corresponds to the first two digits of each firm’s SIC code,  to identify relevant firms. For example, if you are interested in the “Business Service” sector and its "major_group" code is 73, you should retain all firms whose SIC codes start with 73. 

In [ ]:
industries_agri = public_firms[public_firms['sic'].astype(str).str.startswith(agri)]
industries_wholesale = public_firms[public_firms['sic'].astype(str).str.startswith(wholesale)]
industries_food = public_firms[public_firms['sic'].astype(str).str.startswith(food)]
industries_busi_serv = public_firms[public_firms['sic'].astype(str).str.startswith(busi_serv)]
industries_wholesale_only =public_firms[public_firms['sic'].astype(str).str.startswith(wholesale_only)]
fname_industries = industries_wholesale

3.	Now, answer the following questions based on the filtered dataset: 

a.	How many unique firm-year ("fyear") observations are there in the filtered dataset?


In [ ]:
unique_fyear=len(fname_industries['fyear'].unique())
unique_fyear

b.	How many unique firms are there in the filtered dataset?



In [ ]:
unique_firms = len(fname_industries['conm'].unique())
unique_firms

c.	How many firms in the filtered dataset have records over all 27 years (1994-2020)?

In [ ]:
filtered_firms = fname_industries.groupby('conm').filter(lambda x: len(x) == unique_fyear)
num_firms = len(filtered_firms['conm'].unique())
num_firms

B.	[Preliminary Analysis; 20 points] Answer the following questions:

1.	What are the top 10 firms with the highest stock price (column "prcc_c") in the year 2020?


In [ ]:
top_ten_stocks = fname_industries.sort_values('prcc_c', ascending=False).loc[fname_industries['fyear'] == 2020].head(10)
top_ten_stocks[['conm']]

2.	What are the top 10 firms with the highest sales (column "sale") in the entire history of the dataset?

In [ ]:
top_10_firm = fname_industries.groupby('conm').sum(numeric_only=True).sort_values('sale', ascending=False).head(10)
top_10_firm[['sale']]


3.	What is the geographical distribution (column "location") of all the firms? In other words, how many firms are there in each location? Please list the top 10 locations. 

In [ ]:
unique_locations = fname_industries.groupby('location')['conm'].nunique().sort_values(ascending=False)
unique_locations

4.	Create a line chart to show the average stock price (column "prcc_c") in the selected sector(s) across the years. If you have selected multiple sectors, draw multiple lines to show them separately.


In [ ]:
#mean_prcc_c = fname_industries.groupby(['sic', 'fyear'])['prcc_c'].mean()

#for i, industry in enumerate(fname_industries['sic'].unique()):
  #  industry_data = mean_prcc_c.loc[industry]
  #  axes[i].plot(industry_data.index, industry_data.values)
  #  axes[i].set_xlabel('')
  #  axes[i].set_xticks([])
  #  axes[i].set_ylabel('')
  #  axes[i].set_title(f'Industry_{industry}')

mean_prcc_c = fname_industries.groupby(['fyear'])['prcc_c'].mean()
style.use('bmh')

plt.plot(mean_prcc_c.index, mean_prcc_c.values)
plt.xlabel('Year')
plt.ylabel('Mean prcc_c') 
plt.title('Mean prcc_c by Year')
plt.show()
#https://stackoverflow.com/questions/63319572/plotting-two-columns-based-on-multiple-variables-in-another-column
#Helpful to plot prcc_c for different industry

5.	Which firm was affected the most by the 2008 Financial Crisis, as measured by the percentage drop in stock price from 2007 to 2008?

In [ ]:
prcc_c_2008 = fname_industries[fname_industries['fyear'] == 2008]
prcc_c_2007 = fname_industries[fname_industries['fyear'] == 2007]
prcc_merged = prcc_c_2008.merge(prcc_c_2007,
                 how='outer', suffixes=('_x', '_y'), on = 'gvkey')
prcc_merged.rename(columns={'prcc_c_x': 'prccc2008', 'prcc_c_y': 'prccc2007','location_x':'location','conm_x':'conm'}, inplace=True)
prcc_merged.drop(prcc_merged.filter(regex=('_'),).columns, axis=1, inplace=True)
prcc_merged.dropna(inplace=True)
prcc_merged.rename(columns={'prccc2008': 'prcc_c_2008', 'prccc2007': 'prcc_c_2007'}, inplace=True)
prcc_merged['prcc_c_drop'] = (prcc_merged['prcc_c_2008'] - prcc_merged['prcc_c_2007'])*100/prcc_merged['prcc_c_2007']
prcc_merged.sort_values('prcc_c_drop', inplace=True)
prcc_merged.reset_index(drop=True,inplace=True)
most_affected_firm = prcc_merged.iloc[[0]]
most_affected_firm
#https://stackoverflow.com/questions/19125091/pandas-merge-how-to-avoid-duplicating-columns
#Initially used to drop duplicates but code didn't work as expected

In [ ]:
fname_industries.columns

6.	Plot the average Return on Assets (ROA) for the firms located in the “USA” across the years. ROA is calculated as ni/asset.

In [ ]:
fname_industries_roa = fname_industries.copy()
fname_industries_roa['roa'] = fname_industries['ni'] / fname_industries['asset']
usa_roa_yearly_mean_plot = fname_industries_roa[fname_industries_roa['location'] == 'USA'].groupby('fyear')['roa'].mean().plot(legend=True)

Part 2. Text Analysis on the Industry Sector

C.	[Text Cleaning; 10 points] The file "data/2020_10K_item1_full.csv" contains a sample of 5,988 firms and their “item 1” content in their 10-K reports in the year 2020.  Load the dataset as a DataFrame and create a new column containing the cleaned text for each “item1” content. Follow the steps below to clean the text:

1.	Convert all words to lowercase.


In [ ]:
twenty20_10K_item1_clean = twenty20_10K_item1_full.copy()
#twenty20_10K_item1_clean.drop_duplicates(inplace=True)
twenty20_10K_item1_clean['item_1_text'] = twenty20_10K_item1_full['item_1_text'].str.lower()
twenty20_10K_item1_clean

2.	Remove punctuations.

3.	Remove stop words based on the list of English stop words in NLTK.

In [ ]:
translator = str.maketrans('', '', string.punctuation)
sw = stopwords.words('english')
def clean_text(text):
    clean_text = text.lower()
    clean_text = clean_text.translate(translator)
    clean_words = [w for w in clean_text.split() if w not in sw]
    return ' '.join(clean_words)

In [ ]:
try:
    twenty20_10K_item1_clean = pd.read_csv(r'C:\Users\sarda\Desktop\UBC\BAIT 508\Group_Project_BAIT508\data\twenty20_10K_item1_clean.csv')
except:
    twenty20_10K_item1_clean['item_1_clean'] = twenty20_10K_item1_clean['item_1_text'].apply(clean_text).reset_index(drop=True)
    twenty20_10K_item1_clean.drop(['name'],axis = 1,inplace=True)
    twenty20_10K_item1_clean.to_csv(r'C:\Users\sarda\Desktop\UBC\BAIT 508\Group_Project_BAIT508\data\twenty20_10K_item1_clean.csv',index=False)  

#twenty20_10K_item1_clean['item_1_clean'] = twenty20_10K_item1_clean['item_1_text'].apply(clean_text).reset_index(drop=True)
#twenty20_10K_item1_clean.drop(['name'],axis = 1,inplace=True)
#twenty20_10K_item1_clean.to_csv(r'C:\Users\sarda\Desktop\UBC\BAIT 508\Group_Project_BAIT508\data\twenty20_10K_item1_clean.csv',index=False) 
twenty20_10K_item1_clean = pd.read_csv(r'C:\Users\sarda\Desktop\UBC\BAIT 508\Group_Project_BAIT508\data\twenty20_10K_item1_clean.csv')    
twenty20_10K_item1_clean

D.	[Keyword Analysis; 20 points] Conduct keywords analysis on your selected industry sector(s). Follow the steps below to complete the analysis:

1.	Create a new DataFrame that includes only firms in your selected industry sectors. Ensure that you merge the 10-K data with the previous "public_firm.csv" data using an inner join.


In [ ]:
public_firms_2020 = fname_industries[fname_industries['fyear'] == 2020]

In [ ]:
merged_pf_i1_2020 = public_firms_2020.merge(twenty20_10K_item1_clean, how='inner', on='gvkey')
merged_pf_i1_2020.head(2)

2.	Generate the top 10 keywords for each firm based on two different methods: word counts and TF-IDF score.

In [ ]:
def get_keywords_wc_top10(text):
    c = Counter(text.lower().split())
    words = []
    for pair in c.most_common(10):
        words.append(pair[0])
    return ' '.join([x[0] for x in c.most_common(10)])
def get_keywords_tfidf_top10(document_list):
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(document_list)
    feature_names = vectorizer.get_feature_names_out()
    top_keywords = []
    for i in range(len(document_list)):
        feature_index = tfidf_matrix[i, :].nonzero()[1]
        tfidf_scores = zip(feature_index, [tfidf_matrix[i, x] for x in feature_index])
        sorted_tfidf_scores = sorted(tfidf_scores, key=lambda x: x[1], reverse=True)
        top_keywords.append(' '.join([feature_names[i] for i, _ in sorted_tfidf_scores[:10]]))
    return top_keywords


merged_pf_i1_2020['keyword_wc_top10'] = merged_pf_i1_2020.item_1_clean.apply(get_keywords_wc_top10)
merged_pf_i1_2020['keyword_tfidf_top10'] = get_keywords_tfidf_top10(merged_pf_i1_2020.item_1_clean.tolist())

merged_pf_i1_2020.to_csv(r'C:\Users\sarda\Desktop\UBC\BAIT 508\Group_Project_BAIT508\data\merged_pf_i1_2020.csv',index=False)
merged_pf_i1_2020.head(5)

3.	Create two wordclouds to visualize the keywords across all firms in the selected sector: one based on the word count of keywords and another based on the TF-IDF score.

In [ ]:
wordcloud_wc = WordCloud(width=800, height=400,background_color='#b4d5FF',colormap='Dark2', max_font_size=130).generate(' '.join(merged_pf_i1_2020.keyword_wc_top10.tolist()))
plt.figure(figsize=(10,5))
plt.imshow(wordcloud_wc)
plt.savefig(r'C:\Users\sarda\Desktop\UBC\BAIT 508\Group_Project_BAIT508\repo_data\imgs\keyword_wc.png')
plt.axis('off')
plt.show()

In [ ]:
wordcloud_tfidf = WordCloud(width=800, height=400, background_color='#b4d5FF',colormap='Dark2', max_font_size=130).generate(' '.join(merged_pf_i1_2020.keyword_tfidf_top10.tolist()))
plt.figure(figsize=(10,5))
plt.imshow(wordcloud_tfidf)
plt.savefig(r'C:\Users\sarda\Desktop\UBC\BAIT 508\Group_Project_BAIT508\repo_data\imgs\keyword_tfidf.png') # save as PNG file
plt.axis('off')
plt.show()

E.	[Word embedding; 20 points] Train a word2vec model and analyze word similarities.
1.	Train a word2vec model with the full 10-K sample (e.g., "data/2020_10K_item1_full.csv"). Please use the cleaned text (e.g., results from Step C) for training.


In [ ]:
try:
    model = Word2Vec.load(r'C:\Users\sarda\Desktop\UBC\BAIT 508\Group_Project_BAIT508\repo_data\models\word2vec.model')
except:
    sent = [row.split() for row in twenty20_10K_item1_clean['item_1_clean']]
    model = Word2Vec(sent, min_count=1, vector_size=50, workers=3, window=3, sg = 1)
    model.save(r'C:\Users\sarda\Desktop\UBC\BAIT 508\Group_Project_BAIT508\repo_data\models\word2vec.model')
    model = Word2Vec.load(r'C:\Users\sarda\Desktop\UBC\BAIT 508\Group_Project_BAIT508\repo_data\models\word2vec.model')

2.	Manually inspect the wordclouds you generated in D.3 and choose three representative keywords that are relevant to the industry sector of your interest. Utilize the trained word2vec model to find the most relevant five words for each of these three keywords.

In [ ]:
#words_top10_wc = ['distribution','sales','service'] #Related to Wholesale
words_top10_tfidf = ['reseller','sales','service'] #Related to Wholesale

In [ ]:
for _ in words_top10_tfidf:
    print(f'Similar words for {_}: {model.wv.most_similar(positive=_)[:5]}')
    

Part 3. Comprehensive Analysis of One Sample Firm
F.	[Firm Analysis and Strategy Suggestion; 10 points] This is an open question. Pick one firm that you are interested in and try to analyze its market status. The ultimate goal is to provide one valuable suggestion to the firm based on your analysis. Some directions you might consider are, but not limited to:
1.	Convert the keywords extracted in D.2 into word embeddings with the word2vec model trained in E.1. Add up the embeddings for each firm to create the firm-level embeddings. Use the firm-level embeddings to find the focal firm’s competing firms (or, most similar firms). 


In [ ]:
dsimilar = DocumentSimilarity(model = model, gvkeys=merged_pf_i1_2020['gvkey'], conm = merged_pf_i1_2020['conm'], 
                       keywordslist = merged_pf_i1_2020['keyword_tfidf_top10'])
firm = merged_pf_i1_2020.loc[merged_pf_i1_2020['sale'].idxmax(), 'gvkey']
dsimilar.get_firm_embedding(firm)
most_similar_firms_ = dsimilar.most_similar(firm, topn = 5)
most_similar_firms_

2.	Compare the revenue, market share, and ROA of the focal firm to its competitors and provide suggestions accordingly.

In [ ]:
firms = []
for _ in most_similar_firms_:
    firms.append(firm)
    firms.append(_[0])
most_similar_firms = merged_pf_i1_2020[merged_pf_i1_2020['gvkey'].isin(firms)].copy()
most_similar_firms['revenue'] = most_similar_firms['sale']
most_similar_firms['market_share'] = most_similar_firms['sale'] / most_similar_firms['sale'].sum()
most_similar_firms

In [ ]:
columns_to_plot = ['revenue','roa', 'market_share']
for col in columns_to_plot:
    plt.figure(figsize=(15, 5))
    sns.barplot(x=col, y='conm', data=most_similar_firms)
    plt.savefig(f'imgs\\{col}.png')
    plt.show()
    

3.	Perform an analysis of the historical stock prices, ROA, revenue, and assets of the chosen company. Investigate potential correlations and address noteworthy decreases and increases.

In [ ]:
historical_data_for_firm = public_firms[public_firms['gvkey'] == firm].copy()
historical_data_for_firm

sns.lineplot(x='fyear', y='sale', data=historical_data_for_firm)
plt.xlabel('Fiscal Year')
plt.ylabel('Revenue')
plt.title('Historical Revenue for the Firm')
plt.show()

sns.lineplot(x='fyear', y='roa', data=historical_data_for_firm)
plt.xlabel('Fiscal Year')
plt.ylabel('Return on Assets')
plt.title('Historical ROA for the Firm')
plt.show()

sns.lineplot(x='fyear', y='prcc_c', data=historical_data_for_firm)
plt.xlabel('Fiscal Year')
plt.ylabel('Stock Price')
plt.title('Historical Stock Price for the Firm')
plt.show()

sns.lineplot(x='fyear', y='asset', data=historical_data_for_firm)
plt.xlabel('Fiscal Year')
plt.ylabel('Assets')
plt.title('Historical Assets value for the Firm')
plt.show()

sns.heatmap(historical_data_for_firm[['prcc_c', 'ch', 'ni', 'asset', 'sale', 'roa']].corr(numeric_only=True), annot=True)
# reference - https://blog.quantinsti.com/creating-heatmap-using-python-seaborn/#:~:text=A%20heatmap%20is%20a%20two,as%20per%20the%20creator's%20requirement.
# to create heatmap with correlation